In [1]:
# !tar -xf dogs.zip --directory ./dogs

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras import models, layers

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, MaxPooling2D, Flatten, Rescaling, Conv2D, Input
from tensorflow.keras.utils import image_dataset_from_directory as idd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

print(tf.config.list_physical_devices())
GPU = '/GPU:0'
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
MIN_ACCURACY = 0.741
MAX_ACCURACY = 0.854

In [4]:
img_height = 128
img_width = img_height
path = 'dogs'

train_ds = idd(path, batch_size=32, validation_split=0.2, subset='training', seed=123,
               image_size=(img_height, img_width))
num_classes = len(train_ds.class_names)

Found 20578 files belonging to 120 classes.
Using 16463 files for training.


In [5]:
model = Sequential()

model.add(Input(shape=(img_height, img_width, 3)))

model.add(Conv2D(img_width / 2, (10, 10), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(img_width, (7, 7), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(img_width, (4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(img_width * 2, (4, 4), activation='relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

opt = tf.optimizers.Adam(1e-5)
model.compile(optimizer=opt, loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 119, 119, 64)      19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 53, 53, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 26, 26, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 23, 23, 128)       262272    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 11, 11, 128)      0

In [6]:
while True:
    with tf.device(GPU):
        model.fit(train_ds, epochs=1, use_multiprocessing=True)
    with tf.device(GPU):
        t_l, t_a = model.evaluate(train_ds)
    print(f'Loss: {t_l:.2f} \nAccuracy: {t_a:.2f}')
    # if (t_a > MIN_ACCURACY) & (t_a <= MAX_ACCURACY):
    #     break
    # if t_a > MAX_ACCURACY:
    #     raise Exception('Accuracy higher than MAX_ACCURACY')
print(f'Accuracy: {t_a}')

C:\Python310\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


515/515 [==============================] - 8s 15ms/step - loss: 4.7435 - accuracy: 0.0222
Loss: 4.74 
Accuracy: 0.02
515/515 [==============================] - 8s 15ms/step - loss: 4.4410 - accuracy: 0.0513
Loss: 4.44 
Accuracy: 0.05
515/515 [==============================] - 8s 15ms/step - loss: 4.2635 - accuracy: 0.0702
Loss: 4.26 
Accuracy: 0.07
319/515 [=================>............] - ETA: 8s - loss: 4.2345 - accuracy: 0.0752

KeyboardInterrupt: 

In [ ]:
model.save('dogs.h5')

In [ ]:
model = load_model('dogs.h5')
path = '.\\recources'

for clazz in os.listdir(path):
    img_path = os.path.join(path, clazz)
    for img in os.listdir(img_path):
        pic_path = os.path.join(img_path, img)

        img = image.load_img(pic_path, target_size=(img_height, img_width))
        plt.imshow(img)
        plt.show()

        img_array = image.img_to_array(img)
        img_batch = np.expand_dims(img_array, axis=0)
        img_preproc = preprocess_input(img_batch)
        prediction = model.predict(img_preproc)
        pred_class = train_ds.class_names[np.argmax(prediction)]
        print(f'Действительный класс: {clazz}')
        print(f'Предсказанный класс: {pred_class}')
